In [1]:
import json
import sys,os
import pomegranate
import numpy as np
import random
import pandas as pd
import pickle
sys.path.append("../../../")

from cropseq import cfg

In [2]:
# Load dataset sample
ds_path = cfg.resource('dataset.pickle')
df = pd.read_pickle(ds_path)

df_lus = df[["2021","land_usage"]].drop_duplicates()
df_lus.columns = ["code","land_usage"]

columns = ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
X = df[columns].values
y = df['2021'].values
X = X[random.sample(range(0,X.shape[0]), 1000),:] # sample 1000 randomly

In [3]:
# load land-usage code to name mapping
with open(cfg.resource("lu_mapping_2021.json"), 'r') as fout:
    lu_dict = json.load(fout)


In [4]:
# Load trained modelo with 4 hidden states and discrite distribution for observation emission prob. distribution
model = pickle.load(file=open(cfg.resource("hmm.pickle"), 'rb'))
print(model)

None:{
    "class" : "State",
    "distribution" : {
        "class" : "Distribution",
        "dtype" : "numpy.uint8",
        "name" : "DiscreteDistribution",
        "parameters" : [
            {
                "0" : 0.0020748336905820055,
                "2" : 0.0015392168280324607,
                "3" : 0.005114584898380839,
                "4" : 0.0,
                "6" : 0.008877154780308288,
                "7" : 0.00836384063587913,
                "8" : 0.0,
                "10" : 0.0,
                "11" : 0.00922065714098055,
                "12" : 0.0038624502596528248,
                "13" : 0.0021580719583834816,
                "14" : 0.0,
                "15" : 0.0030028059130984444,
                "17" : 0.011794219915670378,
                "20" : 0.05200244774908914,
                "31" : 0.1397498122563102,
                "32" : 0.5393867153551771,
                "33" : 0.0,
                "34" : 0.0,
                "35" : 0.11581615358311985,
            

In [5]:
[x.name for x in model.states]

['s0', 's1', 's2', 's3', 'None-start', 'None-end']

In [6]:
def plot_fw_matrix(sequence, model, lu_dict):
    fw_matrix = np.exp(model.forward(sequence))

    # Display the forward probabilities
    print("         " + "".join(s.name.center(len(s.name)+10) for s in model.states))
    for i in range(len(sequence) + 1):
        print(" <start> " if i==0 else lu_dict[str(sequence[i - 1])].center(9), end="")
        print("".join("{:.5%}".format(fw_matrix[i, j]).center(len(s.name) + 10)
                      for j, s in enumerate(model.states)))

In [7]:
# Lets take one sequence from the dataset and  estimate the emision probability given the model (problem type -1)
sample = X[1,:]
print("Sequence of length {} --> {}\n".format(len(sample),sample))


print("""To calculate the likelihood of an observation sequence from an HMM we can use the forward algorithm.
 The model returns the forward matrix, with the log-likelihood of each state, we have to appli exp. to obtain
 probabilities."""
      )

fw_matrix = np.exp(model.forward(sample))
print("Matrix is size: {}".format(fw_matrix.shape))
# print(fw_matrix)

print("""
The matrix shows per each row the probability of emitting the observed crop in each state.
""")

plot_fw_matrix(sample, model, lu_dict)


print("""
To compute the probability of the sequence we call the method log_probabilityy and again transform the values from log-prob to probability:
""")
probability_percentage = np.exp(model.log_probability(sample))

print("\nThe likelihood over all possible paths " + \
      "of this model producing the sequence {} is {:.10%}\n\n"
      .format(sample, probability_percentage))

Sequence of length 10 --> [32 50 32 32 20 31 20 32 32 20]

To calculate the likelihood of an observation sequence from an HMM we can use the forward algorithm.
 The model returns the forward matrix, with the log-likelihood of each state, we have to appli exp. to obtain
 probabilities.
Matrix is size: (11, 6)

The matrix shows per each row the probability of emitting the observed crop in each state.

              s0          s1          s2          s3          None-start          None-end     
 <start>   0.00000%    0.00000%    0.00000%    0.00000%       100.00000%          0.00000%     
  Barley  52.31606%    0.05927%    0.00000%    0.00000%        0.00000%           0.00000%     
Sunflower  2.13155%    0.05812%    0.00009%    3.28571%        0.00000%           0.00000%     
  Barley   0.64694%    0.00289%    0.45369%    1.18680%        0.00000%           0.00000%     
  Barley   0.19635%    0.00063%    0.33895%    0.40922%        0.00000%           0.00000%     
Bare soil  0.00575%  

In [8]:
# shorter sequences can be used:
#     "32": "Barley",
#     "31": "Wheat",
#     "20": "Bare soil",
sample = [32, 31, 20, 32]
plot_fw_matrix(sample, model, lu_dict)

probability_percentage = np.exp(model.log_probability(sample))

print("\nThe likelihood over all possible paths " + \
      "of this model producing the sequence {} is {:.10%}%\n\n"
      .format(sample, probability_percentage))

              s0          s1          s2          s3          None-start          None-end     
 <start>   0.00000%    0.00000%    0.00000%    0.00000%       100.00000%          0.00000%     
  Barley  52.31606%    0.05927%    0.00000%    0.00000%        0.00000%           0.00000%     
  Wheat    4.11391%    0.08384%    0.00091%    3.10568%        0.00000%           0.00000%     
Bare soil  0.12038%    0.00941%    0.21298%    0.22465%        0.00000%           0.00000%     
  Barley   0.03654%    0.00028%    0.11360%    0.07716%        0.00000%           0.00000%     

The likelihood over all possible paths of this model producing the sequence [32, 31, 20, 32] is 0.2275736435%%




Decoding the most probable hicced state sequence

In [9]:

print("Lets try to give meaning to hidden states by extracting the hidden sequence")

viterbi_likelihood, viterbi_path = model.viterbi(sample)

print("The most likely hidden state sequence to have generated these observations is {} at {:.5%}."
      .format([s[1].name for s in viterbi_path[1:]], np.exp(viterbi_likelihood))
)

Lets try to give meaning to hidden states by extracting the hidden sequence
The most likely hidden state sequence to have generated these observations is ['s0', 's3', 's2', 's2'] at 0.08191%.


In [61]:
print("""
As an use case of this model lets think we want to predict the next land usage for a sequence given the last N years.
For this we have to calculate the probability for all the possible N+1 sequences using forward algorithm and keep the
one with the highest probability.
""")
n_sequence  = [32, 31, 20, 32, 32, 32, 32, 32, 32, 32, 32, 32]
land_usages = lu_dict.keys()

next_sequences = {lu_dict[x]: n_sequence + [int(x)]  for x in land_usages}
# calculate forward prob for each algorith

max_prob = -99999
lu_max_prob = None
total = 0
lu_prob = {}
for lu in sorted(next_sequences):
    try:
        fw_matrix = model.forward(next_sequences[lu])
        lu_prob[lu] = max(np.exp(fw_matrix[-1,:]))
        if lu_prob[lu] > max_prob:
            max_prob = lu_prob[lu]
            lu_max_prob = lu
        total += lu_prob[lu]
    except:
        pass

for lu in sorted(lu_prob):
    print("{}:\t {:.5%}".format(lu, lu_prob[lu]/total))

print("""
Most probable next land usage for sequence {}: <<{}>> with probability {:.5%}
""".format([ lu_dict[str(x)] for x in n_sequence], lu_max_prob, max_prob/total ))
# fw_matrix = np.exp(model.forward(next_sequences[2]))
# print(max(fw_matrix[-1,:]))


As an use case of this model lets think we want to predict the next land usage for a sequence given the last N years.
For this we have to calculate the probability for all the possible N+1 sequences using forward algorithm and keep the
one with the highest probability.

Abandoned woody crops:	 0.00039%
Alfalfa:	 0.29109%
Artificial surfaces:	 0.05254%
Bare soil:	 19.04693%
Barley:	 38.45061%
Beet:	 0.47687%
Bodies of water:	 0.00118%
Broad-leaved deciduous trees:	 0.01375%
Broad-leaved evergreen trees:	 0.00275%
Coniferous trees:	 0.04104%
Fruit trees:	 0.00432%
Grassland:	 0.14377%
Green peas:	 0.44936%
Horticultural crops:	 0.11572%
Maize:	 0.05415%
Nuts trees:	 0.00469%
Oats:	 1.36863%
Olive groves:	 0.00455%
Other cereals:	 0.66689%
Other leguminous:	 0.18433%
Populus plantations :	 0.02294%
Potatoes:	 0.86142%
Rapeseed:	 0.96460%
Ray Grass:	 1.49361%
Rye:	 5.15061%
Scrub:	 0.05359%
Sunflower:	 2.00509%
Uncultivated land :	 5.31745%
Vetch:	 1.52875%
Vineyard:	 0.01227%
Wheat:	 21.

the model is not performing well, after passing 2-3 years the crop is likely to change. The model doesn't take into account the expected change in hidden state.

In [63]:
sequence  = [32, 31, 20, 32, 32]
emission, transition = model.forward_backward(sequence)
emission

array([[1.01188788e+000, 3.60269860e-003, 4.48086807e-063,
        8.74792199e-001, 0.00000000e+000, 0.00000000e+000],
       [1.06733705e-153, 5.94067488e-004, 3.37519503e-003,
        1.82842609e-004, 0.00000000e+000, 0.00000000e+000],
       [8.14260474e-099, 6.20975165e-005, 8.29926346e-001,
        7.38514668e-050, 0.00000000e+000, 0.00000000e+000],
       [2.38634702e-062, 2.30624707e-045, 5.93595614e-001,
        6.81981059e-001, 0.00000000e+000, 0.00000000e+000],
       [9.99648945e-001, 3.51054602e-004, 5.76987688e-097,
        1.01585421e-071, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000]])

In [64]:
transition

array([[-3.51116236e-04, -7.95456879e+00, -2.21598103e+02,
        -1.63467812e+02],
       [-8.23355572e-01, -6.05585773e+00, -8.15968774e+00,
        -5.82655146e-01],
       [-1.32038301e+00, -6.64093959e+00, -1.05298751e+00,
        -9.60327975e-01],
       [-1.68932241e+00, -8.79717088e+00, -7.32285956e-01,
        -1.09543050e+00],
       [-2.10986737e+00, -7.68904959e+00, -5.15991089e-01,
        -1.26805124e+00]])

In [92]:
print("""
Lest try to get the next observation probability using the transition matrix, we first need the most probable final state,
for this we will use the viterbi algorithm
""")

logp, path = model.viterbi(sequence)
last_state = path[-1]

print("Having the final state we just need to get the max value from the discrete distribution parameters")
distrib = last_state[1].distribution.parameters[0]
import operator
max_key = max(distrib.items(), key=operator.itemgetter(1))[0]
print("Next most probable Land usage: <<{}>> with probability: {:.5%}".format(lu_dict[str(max_key)], distrib[max_key]))

print("Same results")


Lest try to get the next observation probability using the trasition matrix, we first need the most probahle final state,
for this we will use the viterbi algorithm

having the final state we just need to get the max value from the discrete distribution parameters
Next most probable Land usage: <<Barley>> with probability: 38.79265%
